In [1]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 108.7 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.2
    Uninstalling httpx-sse-0.4.2:
      Successfully uninstalled httpx-sse-0.4.2


In [2]:
import cohere
co = cohere.ClientV2("WuOIVkz19Iv8kNTzDDce4dYxUHIWuiQPqTw7hCI1") # Get your free API key: https://dashboard.cohere.com/api-keys

In [3]:
{
 "_id": "57241",
 "task": "coherence",
 "src": "Make the text more coherent: It lasted for 60 minutes. It featured the three men taking questions from a studio audience.",
 "tgt": "Lasting for 60 minutes, it featured the three men taking questions from a studio audience."
}

{
 "_id": "69028",
 "task": "clarity",
 "src": "Make the sentence clearer: URLe Lilanga (1934 27 June 2005) was a Tanzanian painter and sculptor, active from the late 1970s and until the early years of the 21st century.",
 "tgt": "URLe Lilanga (1934 27 June 2005) was a Tanzanian painter and sculptor, active from the late 1970s and until the early 21st century."
}

{'_id': '69028',
 'task': 'clarity',
 'src': 'Make the sentence clearer: URLe Lilanga (1934 27 June 2005) was a Tanzanian painter and sculptor, active from the late 1970s and until the early years of the 21st century.',
 'tgt': 'URLe Lilanga (1934 27 June 2005) was a Tanzanian painter and sculptor, active from the late 1970s and until the early 21st century.'}

In [4]:
import json

examples = [
    {
        "messages": [
            {"role": "System", "content": "You help make text smooth."},
            {"role": "User", "content": "Make it coherent: It lasted for 60 minutes. Three men answered questions."},
            {"role": "Chatbot", "content": "It lasted 60 minutes, with three men answering questions."}
        ]
    },
    {
        "messages": [
            {"role": "System", "content": "You improve sentences."},
            {"role": "User", "content": "Make smooth: The project was successful. It finished on time."},
            {"role": "Chatbot", "content": "The project finished on time successfully."}
        ]
    }
]

with open("train_data.jsonl", "w", encoding="utf-8") as f:
    for ex in examples:
        json.dump(ex, f, ensure_ascii=False)
        f.write("\n")


In [5]:
!pip install -q transformers peft accelerate datasets bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 16.7 MB/s eta 0:00:00


In [11]:
from huggingface_hub import login

login()

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [13]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="train_data.jsonl", split="train")

def format_chat(example):
    messages = example["messages"]
    user = next(m["content"] for m in messages if m["role"] == "User")
    bot = next(m["content"] for m in messages if m["role"] == "Chatbot")
    return {"input_text": user, "target_text": bot}

dataset = dataset.map(format_chat)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [14]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424


In [16]:
from transformers import TrainingArguments, Trainer

def tokenize(batch):
    model_inputs = tokenizer(batch["input_text"], truncation=True, padding="max_length", max_length=256)
    labels = tokenizer(batch["target_text"], truncation=True, padding="max_length", max_length=256)["input_ids"]
    model_inputs["labels"] = labels
    return model_inputs

tokenizer.pad_token = tokenizer.eos_token # Add this line to set the pad token

tokenized_ds = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)

args = TrainingArguments(
    output_dir="./llama3-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=5,
    save_steps=50,
    warmup_steps=10,
    bf16=False,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds
)

trainer.train()

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tinhocplus16 (tinhocplus16-university-of-transport-comunication) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [cohere] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss


TrainOutput(global_step=3, training_loss=14.100166320800781, metrics={'train_runtime': 50.9505, 'train_samples_per_second': 0.118, 'train_steps_per_second': 0.059, 'total_flos': 69196792725504.0, 'train_loss': 14.100166320800781, 'epoch': 3.0})

In [17]:
model.save_pretrained("./llama3-lora")
tokenizer.save_pretrained("./llama3-lora")


('./llama3-lora/tokenizer_config.json',
 './llama3-lora/special_tokens_map.json',
 './llama3-lora/chat_template.jinja',
 './llama3-lora/tokenizer.json')

In [20]:
import os

offload_dir = "./offload_dir"
os.makedirs(offload_dir, exist_ok=True)

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from transformers import pipeline
import os

# Tạo thư mục offload
os.makedirs("./offload_dir", exist_ok=True)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", use_fast=False)

# Load base model với 8bit + offload FP32 cho CPU
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    load_in_8bit=True,
    offload_folder="./offload_dir",
    llm_int8_enable_fp32_cpu_offload=True  # <--- Bật offload FP32 xuống CPU
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "./llama3-lora", device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
# Tạo pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50
)

# Thử prompt
prompt = "Make this sentence smoother: He go school every day."
output = pipe(prompt)[0]["generated_text"]
print(output)

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Make this sentence smoother: He go school every day. 
The sentence should be rewritten to make it sound more natural and fluent in English. Here are a few options:

* He goes to school every day.
* He attends school every day.
* He goes to school daily.
* He is a daily
